# MPC test code

**source codes are under /src**

owner: Yi Ju. last updated: 2023/02/11

**contact the owner before you edit this notebook (incl. run)**

**AND very clearly mark where you edit (and why)**

### Debug tips
you can check `mpc.cache` to find the last state before bug occurs

In [1]:
import os
import sys
src_path = sys.path[0].replace("notebooks", "src")

data_path = sys.path[0].replace("notebooks", "data")
print(data_path)

if src_path not in sys.path:
    sys.path.append(src_path)
print(sys.path[0])

out_path = sys.path[0].replace("notebooks", "output")

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

from grid_manager import MPC_op
from data_loader import UCSD_dataloader
from battery_model import Battery_base
from optimizer import Battery_optimizer
from predictor import *


# from ev_manager import EVmanager
# import copy

l:\Coding_project\Github_repo\Energy_grid\data
l:\Coding_project\Github_repo\Energy_grid\notebooks
path get in the utils.py: 


UnboundLocalError: local variable 'data_path' referenced before assignment

In [2]:

bat_params_sample ={
    "bat_capacity": 356,
    "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    "bat_price": 150, # $/kWh (old: 1000, ref: Tesla Powerwall)
    "bat_efficacy": 0.98, 
    "bat_life_0": 3650, # days.
    "bat_cycle_0": 3000, # cycles in lifetime
    # battery degradation params
    "deg_model": "throughput",  
        # valid values: "throughput", "Crate", "rainflow", "DOD"
    #   [1. degradation ~ high C-rate]
    "deg_Crate_thres": (0.25, 0.25, 0.25, 0.25),
    "deg_Crate_lambda": (0.8, 1, 1.5, 2),
    #   [2. degradation ~ large cycle depth]
    "deg_rainflow_thres": (0.2, 0.2, 0.2, 0.4),
    "deg_rainflow_lambda": (0.6, 1, 1.5, 1.8),
    #   [3. degradation ~ low SoE range]
    "deg_DOD_thres": (0.5, 0.2, 0.2, 0.1),
    "deg_DOD_lambda": (1.3, 1.15, 0.85, 0.6),
}

In [3]:
mpc = MPC_op()

# Step 1: load data
mpc.load_data(loader=UCSD_dataloader, 
    tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), delta=0.25,
    bld="Hopkins", pv="Hopkins", ev="OSLER", pv_to_bld=0.5, ev_to_bld=0.2, Pmax=10)

# Step 2: Load historical data
mpc.init_historical_data(loader=UCSD_dataloader,
    tstart=datetime(2019,1,1,0,0), tend=datetime(2019,12,31,23,59), delta=0.25,
    bld="Hopkins", pv="Hopkins", ev="OSLER", pv_to_bld=0.5, ev_to_bld=0.2, Pmax=10)

# Step 3: specify other operational params
optimizer_params = {"strategy": "optimal", "language":"gurobi"}
op_params = {"K": 96,
            "dc_price": 0.6,
            "ev_efficacy": 0.98,
            "energy_price_sell": 0.6, 
            "deg_model_opt": "rainflow",
            "ev_charge_rule": "flex",
            "ev_charge_rule_default": "unif",
            "p_grid_max": "1.5",}
mpc.init_op_params(optimizer=Battery_optimizer, optimizer_params=optimizer_params, delta_0=0.25, **op_params)

# Step 4: specify battery
mpc.init_battery(model=Battery_base, params=bat_params_sample, delta_0=0.25)

# Step 5: initialize predictor
# [Yi, 2023/03/08] modify predictor def
mpc.init_predictor(shortcut="GT")



l:\Coding_project\Github_repo\Energy_grid\src\data_loader.py:342: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  load_bld.index = pd.to_datetime(load_bld.index, infer_datetime_format=True)
l:\Coding_project\Github_repo\Energy_grid\src\data_loader.py:343: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  load_pv.index = pd.to_datetime(load_pv.index, infer_datetime_format=True)
l:\Coding_project\Github_repo\Energy_grid\src\data_loader.py:249: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version 

!!!!!!!!!! MISSING VALUES !!!!!!!!!! || [bld] has [4] missing values
========== NA filled ==========
!!!!!!!!!! MISSING VALUES !!!!!!!!!! || [pv] has [4] missing values
========== NA filled ==========
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 65 sessions


l:\Coding_project\Github_repo\Energy_grid\src\data_loader.py:342: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  load_bld.index = pd.to_datetime(load_bld.index, infer_datetime_format=True)
l:\Coding_project\Github_repo\Energy_grid\src\data_loader.py:343: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  load_pv.index = pd.to_datetime(load_pv.index, infer_datetime_format=True)
l:\Coding_project\Github_repo\Energy_grid\src\data_loader.py:249: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version 

!!!!!!!!!! MISSING VALUES !!!!!!!!!! || [bld] has [4] missing values
========== NA filled ==========
!!!!!!!!!! MISSING VALUES !!!!!!!!!! || [pv] has [4] missing values
========== NA filled ==========
!!!!!!!!!! EV SHORT DURATION !!!!!!!!!! || drop 65 sessions


In [14]:
# Step 6: initialize save_config
folder_path=os.path.join(out_path, "debug_test")
print(folder_path)
mpc.init_save_config(save_fn="rainflow_365_opt2",
    folder_path=os.path.join(out_path, "debug_test"),
    checkpoints="2D", recovery=False, recovery_from=None,
    )

mpc.run(tstart=datetime(2019,1,1,0,0), tend=datetime(2020,1,1,0,0), exe_K=96, save=True)

l:\Coding_project\Github_repo\Energy_grid\output\debug_test
========================= 2019-01-01 00:00:00 =========================
========================= 2019-01-02 00:00:00 =========================
========================= 2019-01-03 00:00:00 =========================
========================= 2019-01-04 00:00:00 =========================
========================= 2019-01-05 00:00:00 =========================
========================= 2019-01-06 00:00:00 =========================
========================= 2019-01-07 00:00:00 =========================
========================= 2019-01-08 00:00:00 =========================
========================= 2019-01-09 00:00:00 =========================
========================= 2019-01-10 00:00:00 =========================
========================= 2019-01-11 00:00:00 =========================
========================= 2019-01-12 00:00:00 =========================
========================= 2019-01-13 00:00:00 =========================
====

In [12]:
mpc.battery_est.params

{'bat_capacity': None,
 'bat_p_max': 3,
 'bat_p_min': 3,
 'bat_price': 150,
 'bat_efficacy': 0.98,
 'bat_life_0': 3650,
 'bat_cycle_0': 3000,
 'deg_model': 'rainflow',
 'deg_Crate_thres': (0.25, 0.25, 0.25, 0.25),
 'deg_Crate_lambda': (0.8, 1, 1.5, 2),
 'deg_rainflow_thres': (0.2, 0.2, 0.2, 0.4),
 'deg_rainflow_lambda': (0.6, 1, 1.5, 1.8),
 'deg_DOD_thres': (0.5, 0.2, 0.2, 0.1),
 'deg_DOD_lambda': (1.3, 1.15, 0.85, 0.6)}